In [1]:
class STNode:
    def __init__(self,d):
        self.data = d
        self.left = self.right = self.mid = None
        self.mult = 0
        self.dlistPtr = None
        self.parent = None

    def bin_search(self, d):
        if self.data == d: return self
        if self.data > d: 
            if self.left == None: return self
            return self.left.bin_search(d)
        if self.data < d: 
            if self.right == None: return self
            return self.right.bin_search(d)
        assert(0) # should not get here    

    def __str__(self):
        return "("+str(self.data)+", "+str(self.mult)+")"   

    def strFromTop(self):
        ptr = self
        s = str(self.data)
        while ptr.parent != None and ptr.parent.mid != ptr:
            ptr = ptr.parent
        if ptr.parent == None: return s
        return ptr.parent.strFromTop()+s
   
    def strTree(self):  
        st = "("+str(self.data)+", "+str(self.mult)+")"
        if self.left == self.mid == self.right == None: return st
        st += " -> ["
        if self.left != None:
            st += self.left.strTree()
        else: st += "□"
        if self.mid != None:
            st += ", "+self.mid.strTree()
        else: st += ", □"
        if self.right != None:
            st += ", "+self.right.strTree()
        else: st += ", □"
        return st + "]"
    
class StringTree:
    def __init__(self):
        self.root = None
        self.size = 0
        self.dlist = DLinkedList()
        
    def __str__(self):
        if self.root == None: return "empty"
        return self.root.strTree()

    def add(self,st):
        if st == "": return
        dlistPtr = self.updateDList(st)
        if self.root == None: 
            self.root = STNode(st[0])
        ptr = self.root
        for i in range(len(st)):
            char = st[i]
            found_node = ptr.bin_search(char)
            if char < found_node.data:
                found_node.left = STNode(char)
                found_node.left.parent = found_node
                ptr = found_node.left
            elif char > found_node.data:
                found_node.right = STNode(char)
                found_node.right.parent = found_node
                ptr = found_node.right
            else:
                ptr = found_node
            if i < len(st)-1:
                if ptr.mid == None:
                    ptr.mid = STNode(st[i+1])
                    ptr.mid.parent = ptr
                ptr = ptr.mid
        ptr.mult += 1
        if ptr.mult == 1: 
            ptr.dlistPtr = dlistPtr
        self.size += 1
    
    def addAll(self,A):
        for x in A: self.add(x)

    def printElems(self):
        ptr = self.dlist.head
        st = ""
        while ptr != None:
            st += ptr.data
            if ptr.next != None:
                st += ", "
            ptr = ptr.next
        print(st)

    def min(self):
        if self.root == None: return None
        return self._min(self.root).strFromTop()

    def _min(self, node):
        ptr = node
        while True: 
            while ptr.left != None: ptr = ptr.left
            if ptr.mult > 0: return ptr 
            ptr = ptr.mid
            
    def max(self):
        if self.root == None: return None
        return self._max(self.root).strFromTop()
    
    def _max(self, node):
        temp_max, ptr = None, node

        while ptr: 
            while ptr.right is not None:
                ptr = ptr.right
            if ptr.mult > 0: 
                temp_max = ptr 
            ptr = ptr.mid
        return temp_max

    def count(self, st): 
        if not st:
            return 0

        ptr = self.root
        length = len(st)
        for i in range(length):
            if ptr is None:
                return 0
            
            char = st[i]
            found_node = ptr.bin_search(char)
            
            if found_node.data != char:
                return 0
            
            ptr = found_node

            if i < (length) - 1:
                ptr = ptr.mid

        return ptr.mult
    
    def updateDList(self, st):
        if self.dlist.length == 0:
            return self.dlist.insertLeft(st,None)
        ptr = self.dlist.head
        while ptr != None and ptr.data < st:
            ptr = ptr.next
        if ptr == None:
            return self.dlist.append(st)
        if ptr.data == st: return None
        return self.dlist.insertLeft(st,ptr)
        
    def pred(self, st): 
        if self.dlist.length == 0:
            return None 
        
        ptr = self.dlist.head

        if ptr.data >= st:
            return None 
        
        while ptr.next is not None and ptr.next.data < st:
            ptr = ptr.next
                
        return ptr.data
    
    def remove(self, st):
        prev_chr_node = None 
        if st:
            ptr = self.root
            for i in range(len(st)):
                if not ptr:
                    break 
                char = st[i]
                found_node = ptr.bin_search(char)
                if found_node.data != char:
                    break 
                ptr = found_node
                if i < (len(st)-1):
                    ptr = ptr.mid
            prev_chr_node = ptr
        
        if not prev_chr_node:
            return 

        self.size -= 1

        if prev_chr_node.mult > 1:
            prev_chr_node.mult = prev_chr_node.mult - 1

        else:
            ptr = prev_chr_node 
            self.dlist.remove(ptr.dlistPtr)
            ptr.dlistPtr = None
            ptr.mult = 0

            while ptr: 
                if ptr.right or ptr.left:
                    self._bin_del(ptr)
                    return
                if ptr.mult > 0 or ptr.mid:
                    return
                if not ptr.parent:
                    self.root = None
                    return
                else:
                    if ptr.parent.mid == ptr:
                        ptr.parent.mid = None
                    elif ptr.parent.left == ptr:
                        ptr.parent.left = None
                    else:
                        ptr.parent.right = None

                    ptr = ptr.parent
            
    def _bin_del(self, ptr):
        if ptr.right and ptr.left:
            min_ptr = lft_prnt_ptr = ptr.right

            if min_ptr.left:
                while min_ptr.left:
                    lft_prnt_ptr = min_ptr
                    min_ptr = min_ptr.left

                lft_prnt_ptr.left = min_ptr.right
                if min_ptr.right:
                    min_ptr.right.parent = lft_prnt_ptr

                min_ptr.right = ptr.right
                ptr.right.parent = min_ptr

            min_ptr.left = ptr.left
            ptr.left.parent = min_ptr

            min_ptr.parent = ptr.parent

            if min_ptr.parent is None: 
                self.root = min_ptr
            else:
                if min_ptr.parent.left == ptr:
                    min_ptr.parent.left = min_ptr
                elif min_ptr.parent.right == ptr:
                    min_ptr.parent.right = min_ptr
                else:
                    min_ptr.parent.mid = min_ptr
                    
        elif ptr.left:
            min_ptr = ptr.left
            min_ptr.parent = ptr.parent
            if min_ptr.parent is None: 
                self.root = min_ptr
            else:
                if min_ptr.parent.left == ptr:
                    min_ptr.parent.left = min_ptr
                elif min_ptr.parent.right == ptr:
                    min_ptr.parent.right = min_ptr
                else:
                    min_ptr.parent.mid = min_ptr
                    
        elif ptr.right:
            min_ptr = ptr.right
            min_ptr.parent = ptr.parent
            if min_ptr.parent is None: 
                self.root = min_ptr
            else:
                if min_ptr.parent.left == ptr:
                    min_ptr.parent.left = min_ptr
                elif min_ptr.parent.right == ptr:
                    min_ptr.parent.right = min_ptr
                else:
                    min_ptr.parent.mid = min_ptr
    
    def updateDList2(self, st):
        if self.dlist.length == 0:
            return self.dlist.insertLeft(st,None)
        elif self.dlist.head.data > st:
            return self.dlist.insertLeft(st,self.dlist.head)  
        elif self.dlist.head.data == st:
            return None 

        ptr = self.root
        ptr_of_pred = None 
        length = len(st)
        for index in range(length):
            char = st[index]
            ptr = ptr.bin_search(char)
            
            if ptr.data != char: 
                closest_ptr = ptr
                while closest_ptr.mid:
                    closest_ptr = closest_ptr.mid
                ptr_of_pred = closest_ptr.dlistPtr
                break 

            if index < (length-1):
                if not ptr.mid:
                    ptr_of_pred = ptr.dlistPtr
                    break 
                ptr = ptr.mid
        
        if not ptr_of_pred:
            while not ptr.dlistPtr:
                ptr = ptr.mid
            
            ptr_of_pred = ptr.dlistPtr.prev

        ptr_to_update_on = ptr_of_pred.next 

        if ptr_to_update_on is None:
            return self.dlist.append(st)
        
        if ptr_to_update_on.data == st:
            return None 
        
        return self.dlist.insertLeft(st, ptr_to_update_on)

class DNode:
    def __init__(self, d, n, p):
        self.data = d
        self.next = n
        self.prev = p

    def __str__(self):
        return str(self.data)
        
class DLinkedList:
    def __init__(self):
        self.head = self.tail = None
        self.length = 0

    def insertLeft(self, d, n):
        self.length += 1
        
        if self.length == 1: 
            self.head = DNode(d, None, None)
            self.tail = self.head
            return self.head

        np = n.prev
        n.prev = DNode(d, n, np)
        if np == None:
            self.head = n.prev
        else:
            np.next = n.prev
        return n.prev

    def append(self, d):
        if self.length == 0:
            return self.insertLeft(d,None)
        self.length += 1
        self.tail.next = DNode(d, None, self.tail)
        self.tail = self.tail.next
        return self.tail
        
    def remove(self, n): 
        self.length -= 1
        if n.prev == None:
            if n.next != None: n.next.prev = None
            self.head = n.next
            return
        if n.next == None:
            n.prev.next = None
            self.tail = n.prev
            return
        n.prev.next = n.next
        n.next.prev = n.prev

    def __str__(self):
        if self.head == None: 
            return "empty"
        st = "-"
        ptr = self.head
        while ptr != None:
            st += "-> "+str(ptr)+" "
            ptr = ptr.next
        return st+"|"

In [2]:
def testprint(t,message,step):
    print("\n"+step,":",message,"tree is:\n",t)
    print("Count 'ca', 'can', 'car', 'cat', 'cats':",t.count("ca"),\
    t.count("can"),t.count("car"),t.count("cat"),t.count("cats"))
    print("Size is:",t.size,", elements: ",end="")
    t.printElems()
    print("Min and Max strings:",t.min(),t.max())
    print("Predecessors: '':",t.pred(""),", can:",t.pred("can"),",\
    car:",t.pred("car"),", cat:",t.pred("cat"),", z:",t.pred("z"))
    
t = StringTree()
t.addAll(["car","can","cat","cat","cat"])
testprint(t,"Initially","0")
t.add("")
testprint(t,"After adding the empty string","1")
t.add("ca")
testprint(t,"After adding 'ca'","2")
t.remove("car")
testprint(t,"After removing 'car'","3")
t.remove("cat"); t.remove("cat")
testprint(t,"After removing 'cat' twice","4")
t.remove("ca"); t.add("cats")
testprint(t,"After removing 'ca' and adding 'cats'","5")
t.remove("can"); t.remove("cats"); t.remove("cat")
testprint(t,"After removing 'can', 'cats' and 'cat'","6")


0 : Initially tree is:
 (c, 0) -> [□, (a, 0) -> [□, (r, 1) -> [(n, 1), □, (t, 3)], □], □]
Count 'ca', 'can', 'car', 'cat', 'cats': 0 1 1 3 0
Size is: 5 , elements: can, car, cat
Min and Max strings: can cat
Predecessors: '': None , can: None ,    car: can , cat: car , z: cat

1 : After adding the empty string tree is:
 (c, 0) -> [□, (a, 0) -> [□, (r, 1) -> [(n, 1), □, (t, 3)], □], □]
Count 'ca', 'can', 'car', 'cat', 'cats': 0 1 1 3 0
Size is: 5 , elements: can, car, cat
Min and Max strings: can cat
Predecessors: '': None , can: None ,    car: can , cat: car , z: cat

2 : After adding 'ca' tree is:
 (c, 0) -> [□, (a, 1) -> [□, (r, 1) -> [(n, 1), □, (t, 3)], □], □]
Count 'ca', 'can', 'car', 'cat', 'cats': 1 1 1 3 0
Size is: 6 , elements: ca, can, car, cat
Min and Max strings: ca cat
Predecessors: '': None , can: ca ,    car: can , cat: car , z: cat

3 : After removing 'car' tree is:
 (c, 0) -> [□, (a, 1) -> [□, (t, 3) -> [(n, 1), □, □], □], □]
Count 'ca', 'can', 'car', 'cat', 'cats': 1 